# COMP500827 机器学习在信息安全中的应用 <br> 第二次作业：支持向量机 <br> 截止日期：2024年12月22日24:00

姓名：，学号：

**作业提交要求**：
- 务必在上面填上自己的姓名和学号
- 提交内容应包括：以jupyter notebook形式编写的Python代码（文件后缀名为.ipynb）、带所有输出结果的jupyter notebook文件对应的PDF文件（注意：不需要把数据打包）
- 所有文件须用zip格式打包成一个压缩包，要求解压即可运行
- 压缩包上传至教学系统（https://class.xjtu.edu.cn/course/74005/homework#/ ）

**作业迟交惩罚制度**：在截止时间前提交的作业/大实验不会遭受任何迟交惩罚。每位学生在整个课程期间有三天的延期时间，可以根据自己的需要使用。在使用这三天延期时，每延迟一天，你的作业/大实验成绩将被扣除15%。你可以根据自己的需要灵活使用这三天延期。例如，你可以将这三天全都用在一个作业/大实验上（本次作业/大实验最多只能获得55%的分数），或者分开使用，每个作业/大实验使用一天延期（每个作业/大实验的最高分为85%）。一旦你使用完这三天延期，之后提交的任何迟交作业将无法获得学分。

In [ ]:
import pandas as pd 
import numpy as np

# Scikit-learn库：SVM、分类指标等
from sklearn import preprocessing
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import GridSearchCV

import itertools, math

# Matplotlib库：用于画图
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

# Seaboarn库：用于数据可视化
import seaborn

%matplotlib inline

# 读取数据

数据集（[Credit Card Fraud Detection](https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud/data)）包含2013年9月由欧洲持卡人通过信用卡进行的284,807笔交易。这些交易发生在两天内，其中共有492笔欺诈交易。该数据集非常不平衡，正例（欺诈交易）仅占所有交易的0.172％。

由于用户信息需要保密，数据集中无法提供有关数据的原始功能和更多背景信息。给定一个交易的信息，我们需要判断这个交易是欺诈交易（标签为1），还是非欺诈交易（标签为0）。

In [ ]:
data = pd.read_csv('./creditcard.csv')
# 二分类任务, Class = 1 (欺诈交易)；Class = 0 (非欺诈交易)
class_names=np.array(['0','1'])

# 数据特征可视化

In [ ]:
# 将数据转化成Pandas DataFrame，便于处理
df = pd.DataFrame(data).fillna(0)

In [ ]:
# 数据的统计特征描述（总和、平均值、方差、最小值、第1四分位数、第2四分位数、第3四分位数和最大值）
df.describe()

In [ ]:
# 将欺诈交易数据筛选出来
df_fraud = df[df['Class'] == 1] 

# 按照时间顺序显式欺诈交易金额
plt.figure(figsize=(8,6))
plt.scatter(df_fraud['Time'], df_fraud['Amount'])
plt.title('Time series of fraud amount', fontsize=20)
plt.xlabel('Time', fontsize=20)
plt.ylabel('Amount', fontsize=20)
plt.xlim([0,175000])
plt.ylim([0,2500])
plt.show()

我们首先可以注意到，时间和欺诈发生的频率没有明显关联。此外，大多数欺诈交易金额较小。

In [ ]:
# 筛选出欺诈交易金额超过1000的交易事件
nb_big_fraud = df_fraud[df_fraud['Amount'] > 1000].shape[0]
print('There are only '+ str(nb_big_fraud) + ' fraud cases where the amount was higher than 1000 over ' + str(df_fraud.shape[0]) + ' frauds')

# 样本数据的不均衡性

In [ ]:
number_fraud = len(data[data.Class == 1])
number_no_fraud = len(data[data.Class == 0])
print('There are only '+ str(number_fraud) + ' fraud cases in the original dataset, while there are ' + str(number_no_fraud) +' non-fraud cases.')

所以，这个数据集是很不平衡的。直接用这些数据训练模型可能会有两类问题：
1. 模型可能会偏向多数类，从而导致预测少数类的性能不佳。这是因为当我们以最小化错误率为目标在这种数据集上训练模型时，多数类会被过度代表，模型倾向于更频繁地预测多数类。
2. 当模型处理新数据时，它可能无法很好地泛化。这是因为该模型是在倾斜的数据集上训练的，可能无法处理测试数据中的不平衡。

**计算分类指标**

In [ ]:
# 绘制Confusion Matrix和计算分类指标
def plot_confusion_matrix(
        cm, classes, y_pred, y,
        title='Confusion matrix',
        cmap=plt.cm.Blues,
        plot_flag=True
    ):
    
    # 计算AUC
    test_fpr, test_tpr, te_thresholds = metrics.roc_curve(y, y_pred)
    auc = metrics.auc(test_fpr, test_tpr)
    
    if plot_flag:
        # 绘制confusion matrix
        fmt = 'd' 
        thresh = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            plt.text(j, i, format(cm[i, j], fmt), fontsize=20,
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

        plt.imshow(cm, interpolation='nearest', cmap=cmap)
        plt.title(title, fontsize=20)
        plt.colorbar()
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes, fontsize=15)
        plt.yticks(tick_marks, classes, fontsize=15)
        plt.tight_layout()
        plt.ylabel('True label', fontsize=20)
        plt.xlabel('Predicted label', fontsize=20)

        # 绘制AUC curve
        fig, ax = plt.subplots(figsize=(6,5))
        plt.grid()
        plt.plot(test_fpr, test_tpr, label=f"AUC={auc:.4f}")
        plt.plot([0,1],[0,1],'g--')
        plt.legend(fontsize=20)
        plt.xticks(fontsize=15)
        plt.yticks(fontsize=15)
        plt.xlabel("False Positive Rate", fontsize=20)
        plt.ylabel("True Positive Rate", fontsize=20)
        plt.title("AUC (ROC curve)", fontsize=20)
        plt.grid(color='black', linestyle='-', linewidth=0.5)
        plt.show()

        print(f"Accuracy: {metrics.accuracy_score(y, y_pred):.4f}, Precision: {metrics.precision_score(y, y_pred):.4f}, Recall: {metrics.recall_score(y, y_pred):.4f}, F1 score: {metrics.f1_score(y, y_pred):.4f}")
        
    return auc

观察不平衡的数据会导致什么问题

In [ ]:
# 为了加速，只取前10000个数据实例用于训练
df_demo_train = df[0:10000]
# 在用于训练的特征数据中，我们抛弃无用的时间特征Time。把交易类别Class作为目标分类标签
X_demo_train = df_demo_train.drop(['Time', 'Class'],axis=1)
y_demo_train = df_demo_train['Class']
X_demo_train = np.asarray(X_demo_train)
y_demo_train = np.asarray(y_demo_train)

In [ ]:
# 10000以后的数据实例都用于测试
df_demo_test_all = df[10000:]
X_demo_test_all = df_demo_test_all.drop(['Time', 'Class'],axis=1)
y_demo_test_all = df_demo_test_all['Class']
X_demo_test_all = np.asarray(X_demo_test_all)
y_demo_test_all = np.asarray(y_demo_test_all)

In [ ]:
# 用默认参数的SVM分类模型
classifier_demo = svm.SVC()
classifier_demo.fit(X_demo_train, y_demo_train)

In [ ]:
# 预测标签，画出性能评估结果
prediction_SVM_demo = classifier_demo.predict(X_demo_test_all)
cm = metrics.confusion_matrix(y_demo_test_all, prediction_SVM_demo)
plot_confusion_matrix(cm,class_names, prediction_SVM_demo, y_demo_test_all)

**样本不平衡解决方法:**
- 过采样（oversampling），增加正样本，使得正、负样本数目接近，然后再进行学习。
- 欠采样（undersampling），去除一些负样本，使得正、负样本数目接近，然后再进行学习

由于我们没有足够的数据去增加正样本，这里我们使用欠采样方法平衡样本。注意：只有当我们能够确保所选的少数样本（在这种情况下是非欺诈交易）能够代表整个数据集中所有非欺诈交易时，我们才能使用欠采样方法。

In [ ]:
# 类似的，我们将全部数据划分为训练集和测试集
# 训练集从前15万个数据实例中构造
df_train_all = df[0:150000]
# 根据标签筛选这15万个交易记录中的欺诈交易和非欺诈交易
df_train_1 = df_train_all[df_train_all['Class'] == 1]
df_train_0 = df_train_all[df_train_all['Class'] == 0]
print('In this partial dataset, we have ' + str(len(df_train_1)) +" fraud cases, so we need to sample a similar number of non-fraud cases")

# 通常来说，应该随机采样负样本（df_sample=df_train_0.sample(300)），这里为了避免随机性影响结果，只取前300个负样本
df_sample=df_train_0[0:300]
df_train = pd.concat([df_train_1, df_sample])

In [ ]:
# 在用于训练的特征数据中，我们抛弃无用的时间特征Time。把交易类别Class作为目标分类标签
X_train = df_train.drop(['Time', 'Class'],axis=1)
y_train = df_train['Class']
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)

In [ ]:
# 前15万个样本之后的数据实例全部用于测试
df_test_all = df[150000:]
X_test_all = df_test_all.drop(['Time', 'Class'],axis=1)
y_test_all = df_test_all['Class']
X_test_all = np.asarray(X_test_all)
y_test_all = np.asarray(y_test_all)

# Q1：用欠采样调整训练集后，观察模型性能有何变化。注意：为公平比较，仍然采用默认参数的SVM分类模型。（50%）

In [ ]:
classifier_default = svm.SVC()
# 填入正确的代码，训练模型

In [ ]:
prediction_SVM = classifier_default.predict(X_test_all)
cm = metrics.confusion_matrix(y_test_all, prediction_SVM)
plot_confusion_matrix(cm,class_names, prediction_SVM, y_test_all)

# 特征重要性分析

In [ ]:
# 计算每一对特征之间的皮尔森相关系数
df_corr = df.corr()

In [ ]:
# 画出相关系数的热力图
plt.figure(figsize=(6,5))
seaborn.heatmap(df_corr, cmap='Blues') 
seaborn.set(font_scale=2,style='white')

plt.title('Heatmap correlation')
plt.show()

我们可以注意到的，大多数特征之间没有相关性。这证实了数据说明中关于这些特征来源的解释：由于保密原因，数据无法提供交易方的原始特征和更多关于数据的背景信息。除开'Time'和'Amount'，其他特征都是通过主成分分析（Principle Component Aanalysis）获得的主成分。

即便如此，这31个特征，每一个特征对提升模型性能都一样有效吗？由于我们关心特征是否有助于我们更加正确地预测交易类型，我们以特征和Class的相关性为标准，来观察不同的特征对模型性能贡献的区别。

In [ ]:
# 获取每个特征和Class特征的相关系数，并将其按照绝对值降序排序
rank = df_corr['Class']
df_rank = pd.DataFrame(rank) 
df_rank = np.abs(df_rank).sort_values(by='Class',ascending=False)
df_rank.dropna(inplace=True)

只用相关性绝对值排名靠前的k个特征，重新训练和评估模型性能

In [ ]:
k = 2
X_train_rank = df_train[df_rank.index[1:k]]
X_train_rank = np.asarray(X_train_rank)

In [ ]:
# 测试数据也相应的只用挑选的k个特征的数据实例
X_test_all_rank = df_test_all[df_rank.index[1:k]]
X_test_all_rank = np.asarray(X_test_all_rank)

In [ ]:
classifier_feature = svm.SVC()
classifier_feature.fit(X_train_rank, y_train)

In [ ]:
prediction_SVM_feature = classifier_feature.predict(X_test_all_rank)
cm = metrics.confusion_matrix(y_test_all, prediction_SVM_feature)
plot_confusion_matrix(cm,class_names, prediction_SVM_feature, y_test_all)

# Q2：模仿“第二章-线性模型”课件的第71页，找出最优的特征（50%）
<div>
<img src="./slides.jpg" width="600"/>
</div>

In [ ]:
def findK():
    auc = []
    # 填入正确的代码
    
    return auc

In [ ]:
auc = findK()

fig, ax = plt.subplots(figsize=(8,6))
plt.grid()
plt.plot(range(0,len(auc)), auc)
plt.xlabel("Number of features", fontsize=20)
plt.ylabel("AUC", fontsize=20)
plt.title("SVM classification results", fontsize=20)
plt.grid(color='black', linestyle='-', linewidth=0.5)
plt.show()

# 类型标签权重优化

在这之前使用的支持向量机（SVM）模型中，每个类别的权重是相同的，这意味着漏检一个欺诈交易与误判一个非欺诈交易一样严重。然而，对于银行来说，它们的目标是尽可能准确地检测到欺诈交易！

实际上，通过修改SVM模型中的```class_weight```参数，我们可以在模型训练阶段选择给哪个类别更多的重视。在本例中，描述欺诈交易的```class_1```事件的权重应该比描述非欺诈交易的```class_0```事件的权重更大。

In [ ]:
classifier_weighted = svm.SVC(class_weight={0:0.3, 1:0.7})
classifier_weighted.fit(X_train_rank, y_train)

In [ ]:
prediction_SVM_weighted = classifier_weighted.predict(X_test_all_rank)
cm = metrics.confusion_matrix(y_test_all, prediction_SVM_weighted)
auc_weighted = plot_confusion_matrix(cm,class_names, prediction_SVM_weighted, y_test_all)

# Q3：模型超参数调优（加分题）

通过筛选特征、改变类别标签权重、改变核函数、改变正则化项权重等等超参数调优方法进一步提高模型性能。具体的设置可以参考SVM的相关超参数[说明](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC)、scikit-learn中基于交叉验证的超参数调优函数[```GridSearchCV```](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)、网络资源、大语言模型等等。

注意事项：
- 不能修改```plot_confusion_matrix```函数；
- 不能修改```y_test_all```；
- 训练集只能从前15万个数据实例中构造，前15万个样本之后的数据实例全部用于测试；
- 加分计算方法：当你得到的最优AUC（记作$x^{*}$）大于调整类型标签权重后得到的AUC（记作$x^{\text{weighted}}$，即```auc_weighted=0.8357```）时，你能得到的加分为$p=\lceil\min\{(x^{*}-x^{\text{weighted}})\times 100, 10\}\rceil$，其中$\lceil\cdot\rceil$为向上取整函数。

In [ ]:
# 可能可以参考的超参数搜索函数
def model_selection(X, y):    
    param_set = [
        {
            'kernel': ['linear'], 
            'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], # [ 2**j for j in range(-10,10) ], 
            'class_weight': [{1:i/100.0, 0:(100-i)/100.0} for i in range(20,100,10)]
        },
    ]
    clf = GridSearchCV(svm.SVC(), param_set, n_jobs = 15, cv = 3)
    clf.fit(X, y)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    return clf

clf = model_selection(X_new, y_train)

In [ ]:
# 实现你的超参数调优方案

prediction_yours = 

In [ ]:
# 输出你的预测结果
cm = metrics.confusion_matrix(y_test_all, prediction_yours)
auc_yours = plot_confusion_matrix(cm,class_names, prediction_yours, y_test_all)

In [ ]:
# 你的附加分
auc_weighted=0.8357
p=math.ceil(min((auc_yours-auc_weighted)*100,10))
p